# BD wrapper tutorial - batch distance estimation

This notebook is intended to guide users through an example case of the BD wrapper to establish distances for a large number of position-position-velocity (PPV) sources.

For more information exceeding this tutorial we recommend taking a look at the following papers and resources:

- For a description about the Bayesian Distance Calculator see:
> - [Reid et al. 2016](https://ui.adsabs.harvard.edu/abs/2016ApJ...823...77R/abstract)
> - [Reid et al. 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...885..131R/abstract)

- For a description about new priors included in the BD wrapper and a test of its performance see: 
> - Riener et al. 2020b (coming to arXiv soon)

See also the ``Parameter_settings.ipynb`` notebook for an overview about and explanation of the main parameters of the BD wrapper.

In [ ]:
# TODO: import necessary libraries

import os
from astropy.table import Table

For this tutorial we will use the first 100 entries of Table 1 from [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) that is located in the `data` directory and is named `RD+09_table1.dat`. This table lists the kinematic distance results for 100 molecular clouds compiled from the Galactic Ring Survey.

Lets take a look at the first ten entries of this table:

In [ ]:
t = Table.read(os.path.join('..', 'data', 'RD+09_table1_sel.dat'), format='ascii')
t[:10]

This table includes five columns giving the id of the cloud (**GRSMC**), its Galactic longitude and latitude position (**GLON**, **GLAT**), its radial velocity (**Vlsr**) and the estimated kinematic distance (**Dist**).

We use this table to illustrate how the distance calculation with the BD wrapper proceeds and compare it to the final distance results we get from the Bayesian approach with the BDC.

We now run the distance calculation with the BD wrapper for the `RD+09_table1_sel.dat` table. For this we run the `example_table_grs_mcs.py` script contained in the `tutorials` directory. Running the following cell displays the content of this script.

In [ ]:
%pycat example_table_grs_mcs.py

For this example we use the BDC v2.4 with the default weights for the priors. With `b.check_for_kda_solutions = True` we use the information from literature distance solutions to inform the prior for the kinematic distance ambiguity (KDA). By default all tables from the `KDA_info` directory are used to compute the prior for the KDA, which also includes the distance information from Table 1 from [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract), for which we want to estimate distances. For this tutorial we set `b.exclude_kda_info_tables = ['Roman-Duval+09']` to exclude Table 1 from [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) for the calculation of the KDA prior.

Now we execute the script in the terminal, by going into the `tutorials` directory and running
```shell
python example_table_grs_mcs.py
```

<div class="alert alert-block alert-warning">
NOTE: Running this script will use 75% of all CPUs on the machine you are running it unless the `use_ncpus` parameter is specified.
</div>

Depending on the number of available CPUs the execution of this script might take a couple of minutes.

### Results

If the script was running successfully it produced a new `results` directory that contains two tables (`RD+09_table1_sel_distance_results.dat`, `RD+09_table1_sel_distance_results_p_max.dat`). Lets first take a look at the first ten entries of the `RD+09_table1_sel_distance_results.dat` table:

In [ ]:
t = Table.read(os.path.join('results', 'RD+09_table1_sel_distance_results.dat'), format='ascii')
t[:10]

This table is an appended version of the original table, in which the additional columns contain the full information on the distance results. The additional columns are:
- **comp**: the number of reported possible distance results for the source. The BDC v2.4 always reports two distance results (`comp = 2`), so the full distance info for each source occupies two rows in the table.
- **dist**: the calculated heliocentric distance in [kpc]
- **e_dist**: the uncertainty in **dist** in [kpc]
- **prob**: the estimated probability for the distance result. 
- **arm**: the associated spiral arm or Galactic feature; `...` denotes no association.
- **c_u, c_v, c_w**: Coordinates centered at the sun. The **c_u** coordiante is along GLON=0°, the **c_v** coordinate is along GLON=90°, the **c_w** coordinate is along GLAT=90°. These three values are all given in [kpc].
- **p_far**: probability that the source is located at the far kinematic distance 
- **KDA_ref**: reference to literature work whose distance(s) helped to inform p_far. See Table A.1 in Riener et al. (2020b) for more information on all literature distance solutions we compare in this step.
- **kDist_1**: near kinematic distance solution in [kpc]
- **kDist_2**: far kinematic distance solution in [kpc]
- **rgal** Galactocentric distance in [kpc]

Note that each source occupies two rows in the distance results table, corresponding to the two reported distance solutions by the BDC v2.4. Note that there could have been more possible distance results than the two reported ones in the table; however the estimated probability of these additional components in the distance probability density function is in the vast majority of cases much lower than the probability of the two reported components. 

In some cases (e.g. for `G018.89-00.51`) there was only one distance component with `prob=1.0`. The BDC still reports a second distance component with `dist=0.0`, which can be ignored.

If the two reported distances were the only distance solutions their combined **prob** values sum up to one. For example, the estimated probability values for `G049.49-00.41` are 90% (for `dist=5.45`) and 10% (for `dist=4.54`).

We can also see the influence of the KDA prior (Sect. 3.2 in Riener et al. 2020b) in the **p_far** and **KDA_ref** columns. For example, the coordinates of `G018.89-00.51` coincided with a clump from the catalogue of [Urquhart et al. (2018)](https://ui.adsabs.harvard.edu/abs/2018MNRAS.473.1059U/abstract) abbreviated as `U+18`. This clump is located at the near kinematic distance, which is why `p_far` was reduced to a value of `0.12`, which weights the kinematic distance solutions so as to favour the near distance. For the `G053.59+00.04` and `G049.49-00.41` clouds no literature sources could be associated (`KDA_ref=--`), which means their `p_far` value stayed at the neutral position of `0.5`, thus neither favoring the near or far kinematic distance solution.

Next, lets take a look at the first five rows of the `RD+09_table1_sel_distance_results_p_max.dat` table:

In [ ]:
t = Table.read(os.path.join('results', 'RD+09_table1_sel_distance_results_p_max.dat'), format='ascii')
t[:5]

This table shows only the final selected distance components for the clouds. Note the additional **flag** column, whose value indicates how the distance was chosen. There are five possible flags for the distance choice:
- (0): the distance assignment yielded only one distance solution
- (1): the associated Gaussian fit of one distance solution did not satisfy the criterion for the amplitude threshold
- (2): the distance solution with the highest probability (i.e. the highest integrated intensity of the associated Gaussian fit) was chosen
- (3): the distance solution with the lowest absolute distance error was chosen
- (4): the near kinematic distance solution was picked randomly

See Sect. 3.4 and Appendix C.3 in Riener et al. 2020b for more details on the choice of the distance solution.

Finally, we can make a scatter plot to compare the literature distances of the 100 molecular clouds with the distances we estimated with the BD wrapper:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(t['dist'], t['Dist'])
ax.set_xlabel('Distance BDC v2.4 [kpc]')
ax.set_ylabel('Distance RD+09 [kpc]')
plt.show()

Our settings for the BD wrapper yielded a very good correspondence with the literature distance results for the majority of sources, even though for about 10% of the clouds the BDC settled on a very different distance solution. Note here that [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) considered the whole PPV extent of the molecular clouds in their distance assignment, whereas the BDC only estimated the distance for the *central* PPV position of the molecular cloud (as given in `RD+09_table1_sel.dat`). [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) also used a different rotation curve model and a different distance to the Galactic center, which both account for small variations in the estimated distances.

### Next steps

You should now be ready to run your own table through the BD wrapper. Check out the `Parameter_settings.ipynb` notebook for more information about possible settings for the BD wrapper.

Try also to change some of the settings in the `example_table_grs_mcs.py` script. For example, instead of 

```python
b.exclude_kda_info_tables = ['Roman-Duval+09']
```

try to set 

```python
b.kda_info_tables = ['Roman-Duval+09']
```

which uses only the distance information from [Roman-Duval et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...699.1153R/abstract) to inform `p_far`. Check how this influences the final distance results obtained with the BDC.

You can also play around with the weights for the priors (`prob_sa`, `prob_ps`, `prob_gl`, `prob_kd`) to check how this influences the distance outcome.

Also take a look at the other tutorial (`Tutorial-plot_distance_pdf.ipynb`) that shows how to plot the distance probability density results obtained by the BDC and how to retain temporary files that can be important for debbuging and obtaining diagnostics of the distance calculation.